<p style="text-align:center; font-size:40px;"><span style="color:orange;">OTP VERIFICATION SYSTEM PROJECT</span></p>


<p style="font-size:25px;"><span style="color:yellow;">Importing Necessary Libraries</span></p>


In [16]:
import random
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import imaplib
import email


<p style="font-size:25px;"><span style="color:yellow;">Function to Generate a 6-Digit OTP</span></p>


In [17]:
def generate_otp():
    return str(random.randint(100000, 999999))


<p style="font-size:25px;"><span style="color:yellow;">Function to send OTP to User's Email Address</span></p>


In [18]:
def send_otp(email, otp):
    # Enter your email credentials here
    email_sender = 'sumis.singh44@gmail.com'
    email_password = 'kvat covv bvhk eddo'

    subject = 'OTP Verification for Education Purpose'
    message = f'Dear Sumit Singh,To complete the verification process your OTP is: {otp}'

    msg = MIMEMultipart()
    msg['From'] = email_sender
    msg['To'] = email
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email_sender, email_password)
        server.sendmail(email_sender, email, msg.as_string())
        server.quit()
        return True
    except Exception as e:
        print("Failed to send OTP:", e)
        return False


<p style="font-size:25px;"><span style="color:yellow;">Function to verify entered OTP and retrieve the actual OTP from Email</span></p>


In [20]:
def verify_otp(otp, entered_otp):
    return otp == entered_otp

# Function to retrieve the actual OTP from the email or database
def retrieve_actual_otp(email_address):
    # Connect to Gmail IMAP server
    mail = imaplib.IMAP4_SSL('imap.gmail.com')
    # Login to the email account
    mail.login('sumis.singh44@gmail.com', 'kvat covv bvhk eddo')
    # Select the inbox folder
    mail.select('inbox')
    # Search for emails from the specified email address
    result, data = mail.search(None, f'(FROM "{email_address}")')
    # Get the latest email
    latest_email_id = data[0].split()[-1]
    # Fetch the email data
    result, email_data = mail.fetch(latest_email_id, '(RFC822)')
    # Parse the email message
    raw_email = email_data[0][1]
    msg = email.message_from_bytes(raw_email)
    # Extract the OTP from the email body
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True).decode('utf-8')
    else:
        body = msg.get_payload(decode=True).decode('utf-8')
    otp = body.split('OTP is: ')[-1].strip()  # Assuming OTP is included in the email body
    
    mail.logout()
    return otp


<p style="font-size:25px;"><span style="color:yellow;">Creating DASH App and Setting Layout </span></p>


In [23]:
# Create Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define layout
app.layout = dbc.Container([
    html.H1("OTP Verification System", className="text-center mt-5"),
    html.Div([
        html.Label("Enter your email address:"),
        dcc.Input(id='email-input', type='email', placeholder='Enter email', className='form-control mb-3'),
        html.Button('Send OTP', id='send-otp-button', n_clicks=0, className='btn btn-primary mb-3'),
        dcc.Input(id='otp-input', type='text', placeholder='Enter OTP', className='form-control mb-3'),
        html.Button('Verify OTP', id='verify-otp-button', n_clicks=0, className='btn btn-success mb-3'),
        html.Div(id='verification-result', className='mt-3')
    ])
], className="mt-5")


# Callback to send OTP and verify OTP
@app.callback(
    [Output('otp-input', 'disabled'),
     Output('send-otp-button', 'disabled'),
     Output('verification-result', 'children')],
    [Input('send-otp-button', 'n_clicks'),
     Input('verify-otp-button', 'n_clicks')],
    [State('email-input', 'value'),
     State('otp-input', 'value')],
    prevent_initial_call=True
)
def send_and_verify_otp(send_clicks, verify_clicks, email, entered_otp):
    global num_attempts  # Use the global variable

    ctx = dash.callback_context
    if not ctx.triggered:
        return True, False, None
    
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
    if triggered_id == 'send-otp-button':
        otp = generate_otp()
        sent = send_otp(email, otp)
        if sent:
            num_attempts = 0  # Reset attempts counter
            return False, True, "An OTP has been sent to your email."
        else:
            return True, False, "Failed to send OTP. Please try again later."
    elif triggered_id == 'verify-otp-button':
        if num_attempts < 3:  # Check if the number of attempts is less than 3
            actual_otp = retrieve_actual_otp(email)
            if verify_otp(actual_otp, entered_otp):
                num_attempts = 0  # Reset attempts counter
                return False, True, "OTP verified. Access granted!"
            else:
                num_attempts += 1  # Increment attempts counter
                return False, True, f"Incorrect OTP. You have {3 - num_attempts} attempts left."
        else:
            return True, True, "You have exceeded the maximum number of attempts."


<p style="font-size:25px;"><span style="color:yellow;">Run the App Code</span></p>


In [24]:
if __name__ == '__main__':
    app.run_server(debug=True)
